In [21]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [22]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)


In [23]:
batchsize=8

In [24]:
train_data= train_datagen.flow_from_directory(r'D:\dataset\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\dataset\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='validation')

Found 64143 images belonging to 2 classes.
Found 16035 images belonging to 2 classes.


In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\dataset\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4720 images belonging to 2 classes.


In [26]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3), batch_size=batchsize))
hmodel = bmodel.output

In [27]:
bmodel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(8, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (8, 39, 39, 32)      864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (8, 39, 39, 32)     96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (8, 39, 39, 32)      0           ['batch_normalization_

Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
__________________________________________________________________________________________________


In [28]:
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(8, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (8, 39, 39, 32)      864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (8, 39, 39, 32)     96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (8, 39, 39, 32)      0           ['batch_normalization_94[0]

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [33]:
checkpoint = ModelCheckpoint(r'D:\dataset\model\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [34]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=2)

<ipython-input-34-c4e3d058002e>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/2
4316/8017 [===============>..............] - ETA: 10:41 - loss: 0.1959 - accuracy: 0.9253

InvalidArgumentError:  Incompatible shapes: [7,64] vs. [8,64]
	 [[node model_1/dropout_1/dropout/Mul_1
 (defined at C:\Users\ASUS\anaconda3\lib\site-packages\keras\layers\core\dropout.py:105)
]] [Op:__inference_train_function_56020]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/dropout_1/dropout/Mul_1:
In[0] model_1/dropout_1/dropout/Mul:	
In[1] model_1/dropout_1/dropout/Cast:

Operation defined at: (most recent call last)
>>>   File "C:\Users\ASUS\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\ASUS\anaconda3\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "C:\Users\ASUS\anaconda3\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
>>>     app

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)